In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
features, labels = np.load('/content/drive/My Drive/ISEP/A3/DEEP LEARNING/Respiration Sound/preprocessed_data/extracted_features.npy'), np.load('/content/drive/My Drive/ISEP/A3/DEEP LEARNING/Respiration Sound/preprocessed_data/labels.npy')
features = np.delete(features, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)
labels = np.delete(labels, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)

In [14]:
le = LabelEncoder()
i_labels = le.fit_transform(labels)
oh_labels = to_categorical(i_labels)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(features, oh_labels, stratify=oh_labels, test_size=0.2, random_state=42)
num_rows, num_columns, num_channels = 40, 862, 1  # input shape

In [16]:
num_labels = oh_labels.shape[1]
filter_size = 2

In [17]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=filter_size, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=filter_size, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 861, 16)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 430, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 430, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 429, 32)       2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 214, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 214, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 213, 64)        8

In [18]:
num_epochs = 250
num_batch_size = 128

callbacks = [
    ModelCheckpoint(
        filepath='mymodel2_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]

In [19]:
model.fit(np.expand_dims(x_train, axis=3), y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(np.expand_dims(x_test, axis=3), y_test), callbacks=callbacks, verbose=1)

Epoch 1/250
6/6 [==============================] - ETA: 0s - loss: 4.1199 - accuracy: 0.7067
Epoch 00001: val_accuracy improved from -inf to 0.86413, saving model to mymodel2_01.h5
6/6 [==============================] - 1s 197ms/step - loss: 4.1199 - accuracy: 0.7067 - val_loss: 1.1306 - val_accuracy: 0.8641
Epoch 2/250
5/6 [========================>.....] - ETA: 0s - loss: 1.6907 - accuracy: 0.7734
Epoch 00002: val_accuracy did not improve from 0.86413
6/6 [==============================] - 0s 61ms/step - loss: 1.7046 - accuracy: 0.7722 - val_loss: 0.8849 - val_accuracy: 0.8641
Epoch 3/250
5/6 [========================>.....] - ETA: 0s - loss: 1.4148 - accuracy: 0.8562
Epoch 00003: val_accuracy did not improve from 0.86413
6/6 [==============================] - 0s 62ms/step - loss: 1.4235 - accuracy: 0.8554 - val_loss: 0.8423 - val_accuracy: 0.8641
Epoch 4/250
5/6 [========================>.....] - ETA: 0s - loss: 1.0790 - accuracy: 0.8406
Epoch 00004: val_accuracy did not improve fro